## **Data Preparation**

In [0]:
from google.colab import drive
import os
from numpy import nan
import pandas as pd
import re

In [4]:
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
BASE_DIR = "/gdrive/My Drive/Firebolt/Bosch/Outputs"

In [0]:
file_path_new = os.path.join(BASE_DIR, "Rectified", "rectified_data.xlsx")
file_path_old = os.path.join(BASE_DIR, "Archive", "bing_collection_x.xlsx")

In [0]:
old_data = pd.read_excel(file_path_old, sheet_name=0)
new_data = pd.read_excel(file_path_new, sheet_name=0)
new_data = new_data[old_data.columns]
new_data["Useful"].replace("No ", "No", inplace=True)
test_set_new = new_data[new_data["Useful"].isna()]
yes_set_new = new_data[new_data["Useful"] == "Yes"]
no_set_new = new_data[new_data["Useful"] == "No"]
no_subset_new = no_set_new.sample(n=yes_set_new.shape[0])

In [0]:
train_set = yes_set_new.append(no_subset_new, ignore_index=True)

In [0]:
test_set_old = old_data[old_data["Useful"].isna()]
yes_set_old = old_data[old_data["Useful"] == "Yes"]
no_set_old = old_data[old_data["Useful"] == "No"]
no_subset_old = no_set_old.sample(n=yes_set_old.shape[0])
train_old = yes_set_old.append(no_subset_old, ignore_index=True)

In [0]:
train_set = train_set.append(train_old, ignore_index=True)
train_set = train_set.sample(frac=1)

In [11]:
test_set_new.replace(nan, "X", inplace=True)
test_set_old.replace(nan, "X", inplace=True)
test_set = test_set_new.append(test_set_old, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [0]:
# required_columns = ["snippet", "Useful"]
# train_set = train_set[required_columns]
# test_set = test_set[required_columns]

In [0]:
extra_chars_regex = r"\d+|\.{2,}|[^\w+|\s+\.\-]"
abbr_sep_regex = r"(\w\.){2,}"
lone_char_regex = r"(\.\s*|\-|\s+s\s+)"
caps_regex = r"[A-Z]+"
extra_space_regex = r"\s{2,}"

In [0]:
def clean_text_logged(string):
    print("string: {}".format(string))
    a_ = re.subn(extra_chars_regex, " ", string)
    print("\tfirst step: {}".format(a_))
    a_ = re.subn(abbr_sep_regex, "", a_[0])
    print("\tsecond step: {}".format(a_))
    a_ = re.subn(lone_char_regex, " ", a_[0])
    print("\tthird step: {}".format(a_))
    caps_list = [chars for chars in re.findall(caps_regex, a_[0]) if
                 len(chars) > 1]
    print(caps_list)
    b = a_[0]
    if caps_list:
        for i, cap_ in enumerate(caps_list):
            b = b.replace(cap_, "")
            print("\t\tstep {}: {}".format(i, b))
    b = re.subn(extra_space_regex, " ", b)[0]
    print("final: {}".format(b.strip().lower()))
    return b.strip().lower()

In [0]:
clean_subset = train_set[:5].snippet.apply(clean_text_logged)

In [14]:
train_set

,name,url,snippet,Useful
123,Young Leaders Program | Pacific Forum,https://www.pacforum.org/programs/pacific-foru...,The Young Leaders Program was established in 2...,No
24,Public Service | International Cooperation and...,http://ice.ssdpp.fudan.edu.cn/public-service-2/,No Announcement posts,Yes
22,Local Government Officials Training Programme ...,http://www.clair.or.jp/e/cooperation/lgotp/lgo...,Local Government Officials Training Programme....,Yes
75,International Programs | FDA,https://www.fda.gov/international-programs,International Programs Learn about the FDA's i...,No
85,Public service - Wikipedia,https://en.m.wikipedia.org/wiki/Public_service,Public service is a service intended to serve ...,No
...,...,...,...,...
102,Center for Leadership Development Federal Exec...,https://www.opm.gov/services-for-agencies/cent...,We created the International Leadership Develo...,Yes
41,Professional Exchanges | U.S. Embassy & Consul...,https://ma.usembassy.gov/education-culture/exc...,"Exchanges for Professionals Every year, the Em...",Yes
15,Two Exchange Alumni Receive Advancing Leaders ...,https://alumni.state.gov/highlight/two-exchang...,The Advancing Leaders Fellowship program recog...,Yes
121,Exchange - University of Western Ontario,https://www.uwo.ca/international/learning/go_a...,Exchange programs are generally available to a...,No


In [98]:
train_set_copy = train_set.copy()
train_set_copy.rename(columns={"snippet": "snippet_actual"}, inplace=True)
clean_snippets = train_set_copy.snippet_actual.apply(clean_text_logged)
# train_set_copy.snippet = clean_snippets
train_set_copy["snippet"] = clean_snippets

string: The Young Leaders Program was established in 2004 to enhance cross-cultural opportunities for young scholars and foreign policy professionals to improve their policy analysis skills at an early juncture in their careers. Participation in the program affords Young Leaders an unparalleled opportunity to build networks, observe the foreign-policy making process, and voice their generation’s ...
	first step: ('The Young Leaders Program was established in   to enhance cross-cultural opportunities for young scholars and foreign policy professionals to improve their policy analysis skills at an early juncture in their careers. Participation in the program affords Young Leaders an unparalleled opportunity to build networks  observe the foreign-policy making process  and voice their generation s  ', 5)
	second step: ('The Young Leaders Program was established in   to enhance cross-cultural opportunities for young scholars and foreign policy professionals to improve their policy analysis

In [99]:
train_set_copy

,name,url,snippet_actual,Useful,snippet
123,Young Leaders Program | Pacific Forum,https://www.pacforum.org/programs/pacific-foru...,The Young Leaders Program was established in 2...,No,the young leaders program was established in t...
24,Public Service | International Cooperation and...,http://ice.ssdpp.fudan.edu.cn/public-service-2/,No Announcement posts,Yes,no announcement posts
22,Local Government Officials Training Programme ...,http://www.clair.or.jp/e/cooperation/lgotp/lgo...,Local Government Officials Training Programme....,Yes,local government officials training programme ...
75,International Programs | FDA,https://www.fda.gov/international-programs,International Programs Learn about the FDA's i...,No,international programs learn about the interna...
85,Public service - Wikipedia,https://en.m.wikipedia.org/wiki/Public_service,Public service is a service intended to serve ...,No,public service is a service intended to serve ...
...,...,...,...,...,...
102,Center for Leadership Development Federal Exec...,https://www.opm.gov/services-for-agencies/cent...,We created the International Leadership Develo...,Yes,we created the international leadership develo...
41,Professional Exchanges | U.S. Embassy & Consul...,https://ma.usembassy.gov/education-culture/exc...,"Exchanges for Professionals Every year, the Em...",Yes,exchanges for professionals every year the emb...
15,Two Exchange Alumni Receive Advancing Leaders ...,https://alumni.state.gov/highlight/two-exchang...,The Advancing Leaders Fellowship program recog...,Yes,the advancing leaders fellowship program recog...
121,Exchange - University of Western Ontario,https://www.uwo.ca/international/learning/go_a...,Exchange programs are generally available to a...,No,exchange programs are generally available to a...


In [0]:
test_set_copy = test_set.copy()
test_set_copy.rename(columns={"snippet": "snippet_actual"}, inplace=True)

In [59]:
clean_snippets_test = test_set_copy.snippet_actual.apply(clean_text_logged)
test_set_copy["snippet"] = clean_snippets_test

string: AIM OF THE PROGRAMME. To energise and equip members of international leadership teams in order that they are enabled to Lead and Work with their teams and organisations in a sustained, faith-filled world during their mandate. Module 1. Leadership of a lived charism in a global context. Transforming experience into authentic leadership. Module 2
	first step: ('AIM OF THE PROGRAMME. To energise and equip members of international leadership teams in order that they are enabled to Lead and Work with their teams and organisations in a sustained  faith-filled world during their mandate. Module  . Leadership of a lived charism in a global context. Transforming experience into authentic leadership. Module  ', 3)
	second step: ('AIM OF THE PROGRAMME. To energise and equip members of international leadership teams in order that they are enabled to Lead and Work with their teams and organisations in a sustained  faith-filled world during their mandate. Module  . Leadership of a lived char

In [81]:
test_set_copy

,name,url,snippet_actual,Useful,snippet
0,International Leadership Development Programme...,https://www.faithandpraxis.org/go.php?a=2a&id=48,AIM OF THE PROGRAMME. To energise and equip me...,X,to energise and equip members of international...
1,Leadership Management International,https://www.lmi-world.com/,"Empowering, rather than overpowering, is the w...",X,empowering rather than overpowering is the way...
2,International Programs - The United States Cen...,https://www.census.gov/internationalprograms,International Programs The U.S. Census Bureau ...,X,international programs the census bureau condu...
3,"Visas for Foreign Government, International Or...",https://travel.state.gov/content/travel/en/us-...,If you are being sent by your government to a ...,X,if you are being sent by your government to a ...
4,AP State Portal,http://www.ap.gov.in/,AP Govt focus on enhancing the Quality of Life...,X,govt focus on enhancing the quality of life of...
...,...,...,...,...,...
363,Study options and entry requirements | The Uni...,https://www.ed.ac.uk/global/study-abroad/study...,The University of Edinburgh is a charitable bo...,X,the university of edinburgh is a charitable bo...
364,Cross Sector Leadership Exchange CSLE | Privat...,https://www.csleuk.org/,CSLE CIC deliver leadership programmes to enco...,X,deliver leadership programmes to encourage col...
365,Dr Gareth McGrath - Director of Parliamentary ...,https://uk.linkedin.com/in/dr-gareth-mcgrath-1...,Since 2011 I have co-delivered a political exc...,X,since i have co delivered a political exchange...
366,Programme content | MBA | London Business School,https://www.london.edu/masters-degrees/mba/pro...,In order to graduate from the programme at one...,X,in order to graduate from the programme at one...


## Analysis

### Adversarial Validation

In [0]:
from scipy import sparse
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import nltk

In [21]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [25]:
stop_words = set(stopwords.words("english"))
print(stop_words)

{'itself', 'am', 'very', 'was', 'on', 'had', 'too', 'your', 'than', 'should', 'as', 'hasn', 'll', 'then', 'that', 'between', 'the', 'been', 'he', 'does', 'about', 'were', 'm', "couldn't", 'i', 'our', 'hers', 'have', 'or', 'mightn', 'shan', "wouldn't", 'not', "aren't", 'her', 'will', 'why', 'o', "mightn't", 'him', "you'd", 'such', 's', "hasn't", 'before', 'again', 'can', "mustn't", 'wouldn', "that'll", 'this', "it's", 'at', 'more', "wasn't", 'how', 'there', 'after', 'yourselves', "needn't", 'yours', 'mustn', "should've", 'his', 'yourself', 'she', 'doing', 'above', 'all', 'through', "you're", 'now', 'weren', 'while', 'who', 'if', 'with', 'isn', 'didn', 'an', "haven't", 'by', 'couldn', 'a', 'you', 'them', 'over', "she's", 've', "won't", 'during', 'for', 'wasn', "didn't", 'which', 'most', 'until', 'himself', "weren't", 'same', 'having', 'ourselves', 'it', 'my', 'did', 'ours', 'whom', 'theirs', 'further', 'and', 'its', 'has', 'from', "don't", 'aren', 'don', 'those', 'hadn', 'when', 'under',

In [0]:
def adversarial_validation(X, Y, n_splits = 10):
    
    # Combine both datasets
    sparse_merge = sparse.vstack((X, Y))
    
    # Label the datasets
    y = np.array([0 for _ in range(X.shape[0])] + [1 for _ in range(Y.shape[0])])
    
    # Do 10 Fold CV 
    kfold = StratifiedKFold(n_splits = n_splits, shuffle = True)

    lr_auc = np.array([])
    rf_auc = np.array([])
    for train_idx, test_idx in kfold.split(sparse_merge, y):
    
        #Run Log Reg
        x_train, y_train = sparse_merge[train_idx], y[train_idx]
        x_test, y_test = sparse_merge[test_idx], y[test_idx]
        
        log_reg = SGDClassifier(loss = 'log')
        log_reg.fit(x_train, y_train)
        y_test_prob = log_reg.predict_proba(x_test)[:,1]
        lr_auc = np.append(lr_auc, roc_auc_score(y_test, y_test_prob))
        # Run RF
        rf = RandomForestClassifier(n_estimators = 100, n_jobs = -1)
        rf.fit(x_train, y_train)
        y_test_prob = rf.predict_proba(x_test)[:,1]
        rf_auc = np.append(rf_auc, roc_auc_score(y_test, y_test_prob))

    
    # Display results
    print('Logisitic Regression AUC : {:.3f}'.format(lr_auc.mean()))
    print('Random Forest AUC : {:.3f}'.format(rf_auc.mean()))

In [0]:
split_ratio = 1 + int(train_set_copy.shape[0] * 0.7)
train_dataset = train_set_copy[:split_ratio]
test_dataset = train_set_copy[split_ratio:]

In [101]:
test_dataset.shape

(42, 5)

In [29]:
bow = CountVectorizer()
x_train = bow.fit_transform(train_dataset.snippet.values)
x_test = bow.transform(test_dataset.snippet.values)
# x_test = shuffle(x_test)
adversarial_validation(x_train, x_test[:40])

Logisitic Regression AUC : 0.398
Random Forest AUC : 0.441


In [0]:
def remove_stop_words(text):
    word_list = text.split(" ")
    filtered_words = [word for word in word_list if word not in stop_words]
    s = " ".join(filtered_words)
    return s

In [31]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, f1_score, accuracy_score, roc_auc_score, confusion_matrix
import seaborn as sns
sns.set_palette("muted")
    

def calc_f1(p_and_r):
    p, r = p_and_r
    return (2*p*r)/(p+r)


# Print the F1, Precision, Recall, ROC-AUC, and Accuracy Metrics 
# Since we are optimizing for F1 score - we will first calculate precision and recall and 
# then find the probability threshold value that gives us the best F1 score

def print_model_metrics(y_test, y_test_prob, confusion = False, verbose = True, return_metrics = False):

    precision, recall, threshold = precision_recall_curve(y_test, y_test_prob, pos_label = 1)
    
    #Find the threshold value that gives the best F1 Score
    best_f1_index = np.argmax([calc_f1(p_r) for p_r in zip(precision, recall)])
    best_threshold, best_precision, best_recall = threshold[best_f1_index], precision[best_f1_index], recall[best_f1_index]
    
    # Calulcate predictions based on the threshold value
    y_test_pred = np.where(y_test_prob > best_threshold, 1, 0)
    
    # Calculate all metrics
    f1 = f1_score(y_test, y_test_pred, pos_label = 1, average = 'binary')
    roc_auc = roc_auc_score(y_test, y_test_prob)
    acc = accuracy_score(y_test, y_test_pred)
    
    
    if confusion:
        # Calculate and Display the confusion Matrix
        cm = confusion_matrix(y_test, y_test_pred)

        plt.title('Confusion Matrix')
        sns.set(font_scale=1.0) #for label size
        sns.heatmap(cm, annot = True, fmt = 'd', xticklabels = ['No', 'Yes'], yticklabels = ['No', 'Yes'], annot_kws={"size": 14}, cmap = 'Blues')# font size

        plt.xlabel('Truth')
        plt.ylabel('Prediction')
        
    if verbose:
        print('F1: {:.3f} | Precision: {:.3f} | Recall: {:.3f} | AUC: {:.3f} | Accuracy: {:.3f} \n'.format(f1, best_precision, best_recall, roc_auc, acc))
    
    if return_metrics:
        return np.array([f1, best_precision, best_recall, roc_auc, acc])
    
    

# Run Simple Log Reg Model and Print metrics
from sklearn.linear_model import SGDClassifier

# Run log reg 10 times and average the result to reduce predction variance
def run_log_reg(train_features, test_features, y_train, y_test,  alpha = 1e-4, confusion = False, return_f1 = False, verbose = True):
    metrics = np.zeros(5)
    for _ in range(10):
        log_reg = SGDClassifier(loss = 'log', alpha = alpha, n_jobs = -1, penalty = 'l2')
        log_reg.fit(train_features, y_train)
        y_test_prob = log_reg.predict_proba(test_features)[:,1]
        metrics += print_model_metrics(y_test, y_test_prob, confusion = confusion, verbose = False, return_metrics = True)
    metrics /=10
    if verbose:
        print('F1: {:.3f} | Precision: {:.3f} | Recall: {:.3f} | AUC: {:.3f} | Accuracy: {:.3f} \n'.format(*metrics))
    if return_f1:
        return f1
    return log_reg


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [32]:

y_train = np.where(train_dataset.Useful.values == 'Yes', 1, 0)
y_test = np.where(test_dataset.Useful.values == 'Yes', 1, 0)

bow = CountVectorizer()
x_train = bow.fit_transform(train_dataset.snippet.values)
x_test = bow.transform(test_dataset.snippet.values)

run_log_reg(x_train, x_test, y_train, y_test)

tfidf = TfidfVectorizer()
x_train = tfidf.fit_transform(train_dataset.snippet.values)
x_test = tfidf.transform(test_dataset.snippet.values)

run_log_reg(x_train, x_test, y_train, y_test)

F1: 0.726 | Precision: 0.710 | Recall: 0.814 | AUC: 0.755 | Accuracy: 0.707 

F1: 0.758 | Precision: 0.802 | Recall: 0.781 | AUC: 0.823 | Accuracy: 0.767 



SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
train_filtered = train_set_copy.copy()
filtered_snippet = train_filtered.snippet.apply(remove_stop_words)
train_filtered.snippet = filtered_snippet
train_filtered.drop_duplicates(subset=["snippet"], inplace=True)
# train_dataset.snippet.iloc[0]

In [0]:
train_filtered

In [0]:
split_ratio = 1 + int(train_filtered.shape[0] * 0.6)
train_new = train_filtered[:split_ratio]
test_new = train_filtered[split_ratio:]

In [0]:
y_train = np.where(train_new.Useful.values == 'Yes', 1, 0)
y_test = np.where(test_new.Useful.values == 'Yes', 1, 0)

bow = CountVectorizer()
x_train = bow.fit_transform(train_new.snippet.values)
x_test = bow.transform(test_new.snippet.values)

run_log_reg(x_train, x_test, y_train, y_test)

tfidf = TfidfVectorizer()
x_train = tfidf.fit_transform(train_new.snippet.values)
x_test = tfidf.transform(test_new.snippet.values)

run_log_reg(x_train, x_test, y_train, y_test)

In [34]:
!pip install pymagnitude

     |████████████████████████████████| 5.4MB 2.4MB/s 
  Created wheel for pymagnitude: filename=pymagnitude-0.1.120-cp36-cp36m-linux_x86_64.whl size=135918206 sha256=3d434fba91a860042434ac0728cd022f487af6c0802c6903aa6f1f9abb63795e
  Stored in directory: /root/.cache/pip/wheels/a2/c7/98/cb48b9db35f8d1a7827b764dc36c5515179dc116448a47c8a1
Successfully built pymagnitude


In [35]:
!wget http://magnitude.plasticity.ai/glove/medium/glove.6B.100d.magnitude

--2020-04-16 19:02:38--  http://magnitude.plasticity.ai/glove/medium/glove.6B.100d.magnitude
Resolving magnitude.plasticity.ai (magnitude.plasticity.ai)... 52.216.81.42
Connecting to magnitude.plasticity.ai (magnitude.plasticity.ai)|52.216.81.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301989888 (288M) [binary/octet-stream]
Saving to: ‘glove.6B.100d.magnitude’

glove.6B.100d.magni 100%[===================>] 288.00M  26.0MB/s    in 10s     

2020-04-16 19:02:48 (27.7 MB/s) - ‘glove.6B.100d.magnitude’ saved [301989888/301989888]



In [0]:
from tqdm import tqdm_notebook
from nltk import word_tokenize
from pymagnitude import *

In [0]:
y_train = np.where(train_dataset.Useful.values == 'Yes', 1, 0)
y_test = np.where(test_dataset.Useful.values == 'Yes', 1, 0)

In [38]:

glove = Magnitude("glove.6B.100d.magnitude")
def avg_glove(df):
    vectors = []
    for text in tqdm_notebook(df.snippet.values):
        vectors.append(np.average(glove.query(word_tokenize(text)), axis = 0))
    return np.array(vectors)
x_train = avg_glove(train_dataset)
x_test = avg_glove(test_dataset)
run_log_reg(x_train, x_test, y_train, y_test)
# run_log_reg(x_test, x_train, y_test, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



F1: 0.738 | Precision: 0.753 | Recall: 0.786 | AUC: 0.785 | Accuracy: 0.738 



SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [102]:
train_dataset

,name,url,snippet_actual,Useful,snippet
123,Young Leaders Program | Pacific Forum,https://www.pacforum.org/programs/pacific-foru...,The Young Leaders Program was established in 2...,No,the young leaders program was established in t...
24,Public Service | International Cooperation and...,http://ice.ssdpp.fudan.edu.cn/public-service-2/,No Announcement posts,Yes,no announcement posts
22,Local Government Officials Training Programme ...,http://www.clair.or.jp/e/cooperation/lgotp/lgo...,Local Government Officials Training Programme....,Yes,local government officials training programme ...
75,International Programs | FDA,https://www.fda.gov/international-programs,International Programs Learn about the FDA's i...,No,international programs learn about the interna...
85,Public service - Wikipedia,https://en.m.wikipedia.org/wiki/Public_service,Public service is a service intended to serve ...,No,public service is a service intended to serve ...
...,...,...,...,...,...
63,Teaching the American Civil Rights Movement in...,https://alumni.state.gov/alumni-story/teaching...,"Then in 2009, two years after his participatio...",No,then in two years after his participation in r...
115,Robert Bosch Foundation Fellowship Program - C...,https://culturalvistas.org/programs/abroad/rob...,"A fully-funded, 9-to-12 month professional dev...",Yes,a fully funded to month professional developme...
87,Leadership development - Wikipedia,https://en.wikipedia.org/wiki/Leadership_devel...,"Traditionally, leadership development has focu...",No,traditionally leadership development has focus...
137,Latest Fellowships and other opportunities,https://www2.fundsforngos.org/category/fellows...,The Atlas Corps Fellowship is a 12-18 month Fe...,No,the atlas corps fellowship is a month fellowsh...


In [104]:
### Selected!
tfidf = TfidfVectorizer()
tfidf.fit(train_dataset.snippet.values)
# Now lets create a dict so that for every word in the corpus we have a corresponding IDF value
idf_dict = dict(zip(tfidf.get_feature_names(), tfidf.idf_))
# Same as Avg Glove except instead of doing a regular average, we'll use the IDF values as weights.
def tfidf_glove(df):
    vectors = []
    for text in tqdm_notebook(df.snippet.values):
        glove_vectors = glove.query(word_tokenize(text))
        weights = [idf_dict.get(word, 1) for word in word_tokenize(text)]
        vectors.append(np.average(glove_vectors, axis = 0, weights = weights))
    return np.array(vectors)
x_train = tfidf_glove(train_dataset)
x_test = tfidf_glove(test_dataset)
run_log_reg(x_train, x_test, y_train, y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



F1: 0.761 | Precision: 0.735 | Recall: 0.852 | AUC: 0.822 | Accuracy: 0.748 



SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [105]:
log_reg = SGDClassifier()
log_reg.fit(x_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [106]:
test_set_copy

,name,url,snippet_actual,Useful,snippet
0,International Leadership Development Programme...,https://www.faithandpraxis.org/go.php?a=2a&id=48,AIM OF THE PROGRAMME. To energise and equip me...,X,to energise and equip members of international...
1,Leadership Management International,https://www.lmi-world.com/,"Empowering, rather than overpowering, is the w...",X,empowering rather than overpowering is the way...
2,International Programs - The United States Cen...,https://www.census.gov/internationalprograms,International Programs The U.S. Census Bureau ...,X,international programs the census bureau condu...
3,"Visas for Foreign Government, International Or...",https://travel.state.gov/content/travel/en/us-...,If you are being sent by your government to a ...,X,if you are being sent by your government to a ...
4,AP State Portal,http://www.ap.gov.in/,AP Govt focus on enhancing the Quality of Life...,X,govt focus on enhancing the quality of life of...
...,...,...,...,...,...
363,Study options and entry requirements | The Uni...,https://www.ed.ac.uk/global/study-abroad/study...,The University of Edinburgh is a charitable bo...,X,the university of edinburgh is a charitable bo...
364,Cross Sector Leadership Exchange CSLE | Privat...,https://www.csleuk.org/,CSLE CIC deliver leadership programmes to enco...,X,deliver leadership programmes to encourage col...
365,Dr Gareth McGrath - Director of Parliamentary ...,https://uk.linkedin.com/in/dr-gareth-mcgrath-1...,Since 2011 I have co-delivered a political exc...,X,since i have co delivered a political exchange...
366,Programme content | MBA | London Business School,https://www.london.edu/masters-degrees/mba/pro...,In order to graduate from the programme at one...,X,in order to graduate from the programme at one...


In [107]:
x_t = tfidf_glove(test_set_copy[["snippet", "name"]])
# test_dataset

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [0]:
predicted_output = log_reg.predict(x_t)
# predicted_output

In [0]:
output_string = np.where(predicted_output == 1, "Yes", "No")
# output_string

In [110]:
output_string

array(['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes',
       'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes',
       'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes',
       'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes',
       'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes',
       'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes',
       'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes',
       'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes',
       'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'No',
       'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes',
       'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes',
       'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes',
       'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes',
       'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes',
 

In [111]:
test_set_copy_1 = test_set_copy.copy()
test_set_copy_1["useful_predicted"] = output_string
test_set_copy_1.drop(columns=["Useful", "snippet"], inplace=True)
test_set_copy_1

,name,url,snippet_actual,useful_predicted
0,International Leadership Development Programme...,https://www.faithandpraxis.org/go.php?a=2a&id=48,AIM OF THE PROGRAMME. To energise and equip me...,Yes
1,Leadership Management International,https://www.lmi-world.com/,"Empowering, rather than overpowering, is the w...",Yes
2,International Programs - The United States Cen...,https://www.census.gov/internationalprograms,International Programs The U.S. Census Bureau ...,Yes
3,"Visas for Foreign Government, International Or...",https://travel.state.gov/content/travel/en/us-...,If you are being sent by your government to a ...,Yes
4,AP State Portal,http://www.ap.gov.in/,AP Govt focus on enhancing the Quality of Life...,Yes
...,...,...,...,...
363,Study options and entry requirements | The Uni...,https://www.ed.ac.uk/global/study-abroad/study...,The University of Edinburgh is a charitable bo...,No
364,Cross Sector Leadership Exchange CSLE | Privat...,https://www.csleuk.org/,CSLE CIC deliver leadership programmes to enco...,Yes
365,Dr Gareth McGrath - Director of Parliamentary ...,https://uk.linkedin.com/in/dr-gareth-mcgrath-1...,Since 2011 I have co-delivered a political exc...,Yes
366,Programme content | MBA | London Business School,https://www.london.edu/masters-degrees/mba/pro...,In order to graduate from the programme at one...,No


In [194]:
run_log_reg(x_test, x_train, y_test, y_train)

F1: 0.777 | Precision: 0.766 | Recall: 0.814 | AUC: 0.775 | Accuracy: 0.777 



SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
words_features = ["student", "graduate", "study", "grads", "school", "course",
                "internship", "entertainment", "college", "scholarship",
                "bachelor", "scholarly", "academic", "youth", "news", "job"]

In [0]:
def add_feature_relevance(data_df):
    relevance = list()
    df = data_df.copy()
    for text in df.snippet:
        is_bad_keyword_absent = 1
        presence = [word for word in words_features if word in text]
        if presence:
            is_bad_keyword_absent = 0
        relevance.append(is_bad_keyword_absent)
    df["is_relevant"] = relevance
    return df


In [0]:
train_new_set = add_feature_relevance(train_dataset)
test_new_set = add_feature_relevance(test_dataset)

In [92]:
tfidf = TfidfVectorizer()
tfidf.fit(train_new_set.snippet.values)
# Now lets create a dict so that for every word in the corpus we have a corresponding IDF value
idf_dict = dict(zip(tfidf.get_feature_names(), tfidf.idf_))
# Same as Avg Glove except instead of doing a regular average, we'll use the IDF values as weights.
def tfidf_glove(df):
    vectors = []
    for text in tqdm_notebook(df.snippet.values):
        glove_vectors = glove.query(word_tokenize(text))
        weights = [idf_dict.get(word, 1) for word in word_tokenize(text)]
        vectors.append(np.average(glove_vectors, axis = 0, weights = weights))
    return np.array(vectors)
x_train = tfidf_glove(train_new_set)
x_test = tfidf_glove(test_new_set)
run_log_reg(x_train, x_test, y_train, y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



F1: 0.759 | Precision: 0.701 | Recall: 0.895 | AUC: 0.817 | Accuracy: 0.731 



SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [91]:
test_set_copy

,name,url,snippet_actual,Useful,snippet
0,International Leadership Development Programme...,https://www.faithandpraxis.org/go.php?a=2a&id=48,AIM OF THE PROGRAMME. To energise and equip me...,X,to energise and equip members of international...
1,Leadership Management International,https://www.lmi-world.com/,"Empowering, rather than overpowering, is the w...",X,empowering rather than overpowering is the way...
2,International Programs - The United States Cen...,https://www.census.gov/internationalprograms,International Programs The U.S. Census Bureau ...,X,international programs the census bureau condu...
3,"Visas for Foreign Government, International Or...",https://travel.state.gov/content/travel/en/us-...,If you are being sent by your government to a ...,X,if you are being sent by your government to a ...
4,AP State Portal,http://www.ap.gov.in/,AP Govt focus on enhancing the Quality of Life...,X,govt focus on enhancing the quality of life of...
...,...,...,...,...,...
363,Study options and entry requirements | The Uni...,https://www.ed.ac.uk/global/study-abroad/study...,The University of Edinburgh is a charitable bo...,X,the university of edinburgh is a charitable bo...
364,Cross Sector Leadership Exchange CSLE | Privat...,https://www.csleuk.org/,CSLE CIC deliver leadership programmes to enco...,X,deliver leadership programmes to encourage col...
365,Dr Gareth McGrath - Director of Parliamentary ...,https://uk.linkedin.com/in/dr-gareth-mcgrath-1...,Since 2011 I have co-delivered a political exc...,X,since i have co delivered a political exchange...
366,Programme content | MBA | London Business School,https://www.london.edu/masters-degrees/mba/pro...,In order to graduate from the programme at one...,X,in order to graduate from the programme at one...


In [93]:
log_reg = SGDClassifier(loss='log', alpha=1e-4, n_jobs=-1, penalty='l2')
log_reg.fit(x_train, y_train)

x_t = tfidf_glove(test_set_copy)
predicted_output = log_reg.predict(x_t)
output_string = np.where(predicted_output == 1, "Yes", "No")

test_set_copy_2 = test_set_copy.copy()
test_set_copy_2["useful_predicted"] = output_string
test_set_copy_2.drop(columns=["Useful", "snippet"], inplace=True)
test_set_copy_2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


,name,url,snippet_actual,useful_predicted
0,International Leadership Development Programme...,https://www.faithandpraxis.org/go.php?a=2a&id=48,AIM OF THE PROGRAMME. To energise and equip me...,No
1,Leadership Management International,https://www.lmi-world.com/,"Empowering, rather than overpowering, is the w...",No
2,International Programs - The United States Cen...,https://www.census.gov/internationalprograms,International Programs The U.S. Census Bureau ...,No
3,"Visas for Foreign Government, International Or...",https://travel.state.gov/content/travel/en/us-...,If you are being sent by your government to a ...,No
4,AP State Portal,http://www.ap.gov.in/,AP Govt focus on enhancing the Quality of Life...,No
...,...,...,...,...
363,Study options and entry requirements | The Uni...,https://www.ed.ac.uk/global/study-abroad/study...,The University of Edinburgh is a charitable bo...,No
364,Cross Sector Leadership Exchange CSLE | Privat...,https://www.csleuk.org/,CSLE CIC deliver leadership programmes to enco...,No
365,Dr Gareth McGrath - Director of Parliamentary ...,https://uk.linkedin.com/in/dr-gareth-mcgrath-1...,Since 2011 I have co-delivered a political exc...,No
366,Programme content | MBA | London Business School,https://www.london.edu/masters-degrees/mba/pro...,In order to graduate from the programme at one...,No


In [96]:
test_set_copy_2[test_set_copy_2.useful_predicted == "Yes"]

,name,url,snippet_actual,useful_predicted
10,International Urban Cooperation | Root,https://iuc.eu/,International IUC Asia (中文) IUC India (हिन्दी)...,Yes
33,CENTER ON INTERNATIONAL COOPERATION,https://www.odi.org/sites/odi.org.uk/files/odi...,more necessary in meeting these challenges. Th...,Yes
34,South Asian Association for Regional Cooperati...,https://en.m.wikipedia.org/wiki/South_Asian_As...,The South Asian Association for Regional Coope...,Yes
52,Introducing the 2019 Tutu Leadership Programme...,https://alinstitute.org/news/introducing-the-c...,The African Leadership Institute has a very st...,Yes
98,Programs | Meridian International Center,https://www.meridian.org/programs/,Our programs: Build global leaders who have a ...,Yes
99,Education Exchange Programs offered by the U.S...,https://xk.usembassy.gov/education-culture/exc...,The Fulbright Program is the flagship internat...,Yes
100,International Opportunities and Funding Source...,http://www.ala.org/rt/irrt/irrtcommittees/irrt...,List of opportunities and funding resources fo...,Yes
129,Global Leadership Fellows | World Economic Forum,https://www.weforum.org/communities/global-lea...,It’s about empowering widespread innovation an...,Yes
130,Asia Foundation Development Fellows | The Asia...,http://asiafoundation.org/what-we-do/asia-foun...,The deadline for the 2020 Asia Foundation Deve...,Yes
132,Mandela Washington Fellowship | YALI Network,https://yali.state.gov/mwf/,The Mandela Washington Fellowship for Young Af...,Yes


In [0]:
# tfidf = TfidfVectorizer()
# x_train = tfidf.fit_transform(train_new_set.snippet.values)
# x_test = tfidf.transform(test_new_set.snippet.values)

# run_log_reg(x_train, x_test, y_train, y_test)
"""F1: 0.753 | Precision: 0.781 | Recall: 0.786 | AUC: 0.783 | Accuracy: 0.745"""

In [0]:
# train_dataset.drop(columns=["is_relevant"], inplace=True)

In [0]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

### HyperParameter Tuning

In [113]:
!pip install textstat

     |████████████████████████████████| 3.0MB 3.9MB/s 


In [122]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import textstat
from numpy import hstack
import string
import multiprocessing
from sklearn.preprocessing import Normalizer, StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler


def readability_scores_mp(data):
    result_dict, idx, text = data

    flesch_kincaid_grade =  textstat.flesch_kincaid_grade(text)
    dale_chall_readability_score =  textstat.dale_chall_readability_score(text) 

    result_dict[idx] = [flesch_kincaid_grade, dale_chall_readability_score]


def calc_readability_scores(df):
    manager = multiprocessing.Manager()
    result_dict = manager.dict()
    mp_list = [(result_dict, idx, title) for idx, title in enumerate(df.snippet.values)]

    with multiprocessing.Pool(os.cpu_count()) as p:
        r = list(tqdm_notebook(p.imap(readability_scores_mp, mp_list), total=len(mp_list)))
    rows = [result_dict[idx] for idx in range(df.snippet.values.shape[0])]
    return pd.DataFrame(rows).values


def text_features(df):
    longest_word_length = []
    mean_word_length = []
    length_in_chars = []

    for title in df.snippet.values:
        length_in_chars.append(len(title))
        longest_word_length.append(len(max(title.split(), key = len)))
        mean_word_length.append(np.mean([len(word) for word in title.split()]))

    longest_word_length = np.array(longest_word_length).reshape(-1,1)
    mean_word_length = np.array(mean_word_length).reshape(-1,1)
    length_in_chars =  np.array(length_in_chars).reshape(-1,1)
    
    return np.concatenate([longest_word_length, mean_word_length, length_in_chars], axis = 1)


def word_ratio(df):
    with open('DaleChallEasyWordList.txt') as f:
        easy_words_list = [line.rstrip(' \n') for line in f]
    
    with open('terrier-stopword.txt') as f:
        terrier_stopword_list = [line.rstrip(' \n') for line in f]

    terrier_stopword_list += stopwords.words('english')

    with open('common.txt') as f:
        common = [line.rstrip(' \n') for line in f]

    terrier_stopword_list += common

    with open('contractions.txt') as f:
        contractions_list = [line.rstrip(' \n') for line in f]

    with open('hyperbolic.txt') as f:
        hyperbolic_list = [line.rstrip(' \n') for line in f]
    easy_words_ratio = []
    stop_words_ratio = []
    contractions_ratio = []
    hyperbolic_ratio = []
    for title in df.snippet.values:
        easy_words = 0 
        stop_words = 0
        total_words = 0
        contracted_words = 0
        hyperbolic_words = 0
        for word in title.split():
            if word.lower() in easy_words_list:
                easy_words += 1
            if word.lower() in terrier_stopword_list:
                stop_words += 1
            if word.lower() in contractions_list:
                contracted_words += 1
            if word.lower() in hyperbolic_list:
                hyperbolic_words += 1
            total_words += 1
        easy_words_ratio.append(easy_words/total_words)
        stop_words_ratio.append(stop_words/total_words)
        contractions_ratio.append(contracted_words/total_words)
        hyperbolic_ratio.append(hyperbolic_words/total_words)
    easy_words_ratio = np.array(easy_words_ratio).reshape(-1,1)
    stop_words_ratio = np.array(stop_words_ratio).reshape(-1,1)
    contractions_ratio = np.array(contractions_ratio).reshape(-1,1)
    hyperbolic_ratio = np.array(hyperbolic_ratio).reshape(-1,1)
    return np.concatenate([easy_words_ratio, stop_words_ratio, contractions_ratio, hyperbolic_ratio], axis = 1)


def calc_sentiment_scores(df):
    sid = SentimentIntensityAnalyzer()
    neg = []
    neu = []
    pos = []
    compound = []

    for title in df.snippet.values:
        sentiments = sid.polarity_scores(title)
        neg.append(sentiments['neg'])
        neu.append(sentiments['neu'])
        pos.append(sentiments['pos'])
        compound.append(sentiments['compound'])

    neg = np.array(neg).reshape(-1,1)
    neu = np.array(neu).reshape(-1,1)
    pos = np.array(pos).reshape(-1,1)
    compound = np.array(compound).reshape(-1,1)
    return np.concatenate([neg,  pos, compound], axis = 1)


def get_glove_vectors(df, glove):
    vectors = []
    for title in tqdm_notebook(df.snippet.values):
        vectors.append(np.average(glove.query(word_tokenize(title)), axis = 0))
    return np.array(vectors)


def tfidf_w2v(df, idf_dict, glove):
    vectors = []
    for title in tqdm_notebook(df.snippet.values):
        w2v_vectors = glove.query(word_tokenize(title))
        weights = [idf_dict.get(word, 1) for word in word_tokenize(title)]
        vectors.append(np.average(w2v_vectors, axis = 0, weights = weights))
    return np.array(vectors)


In [0]:
def featurize(train_df, test_df, embedding_type):
    
    print('Text Features....')
    train_text_features = text_features(train_df)
    test_text_features = text_features(test_df)

    print('Word ratios....')
    train_word_ratio = word_ratio(train_df)
    test_word_ratio = word_ratio(test_df)

    print('Sentiment Scores....')
    train_sentiment = calc_sentiment_scores(train_df)
    test_sentiment = calc_sentiment_scores(test_df)

    print('Readability Scores....')
    train_readability_scores = calc_readability_scores(train_df)
    test_readability_scores = calc_readability_scores(test_df)
    if embedding_type == 'tfidf_glove':
        print('Glove.....')
        
        glove = Magnitude("glove.6B.100d.magnitude")
        
        tfidf = TfidfVectorizer()
        tfidf.fit(train_df.snippet.values)
        idf_dict = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

        train_glove = tfidf_w2v(train_df, idf_dict, glove)
        test_glove = tfidf_w2v(test_df, idf_dict, glove)

        normalizer_glove = MinMaxScaler()
        train_glove = normalizer_glove.fit_transform(train_glove)
        test_glove = normalizer_glove.transform(test_glove)


        train_embedding_features = sparse.csr_matrix(train_glove)
        test_embedding_features = sparse.csr_matrix(test_glove)
    train_features = hstack((train_text_features, train_word_ratio, train_sentiment, train_readability_scores))
    normalizer = MinMaxScaler()
    train_features = normalizer.fit_transform(train_features)
    train_features = sparse.csr_matrix(train_features)
    train_features = sparse.hstack((train_features, train_embedding_features))
    test_features = hstack((test_text_features, test_word_ratio, test_sentiment, test_readability_scores))
    test_features = normalizer.transform(test_features)
    test_features = sparse.csr_matrix(test_features)
    test_features = sparse.hstack((test_features, test_embedding_features))
    feature_names = [
                     'longest_word_length',
                     'mean_word_length',
                     'length_in_chars',
                     'easy_words_ratio',
                     'stop_words_ratio',
                     'contractions_ratio',
                     'hyperbolic_ratio',
                     'sentiment_neg',
                     'senitment_pos',
                     'sentiment_compound',
                     'flesch_kincaid_grade',
                     'dale_chall_readability_score'
    ]
    feature_names = feature_names + ['glove_' + str(col) for col in range(100)]
    return train_features, test_features, feature_names

In [131]:
train_features, test_features, feature_names = featurize(train_dataset, test_dataset, 'tfidf_glove')
run_log_reg(train_features, test_features, y_train, y_test, alpha = 5e-2)

Text Features....
Word ratios....
Sentiment Scores....
Readability Scores....


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Glove.....


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



F1: 0.729 | Precision: 0.879 | Recall: 0.681 | AUC: 0.800 | Accuracy: 0.764 



SGDClassifier(alpha=0.05, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer



# Create a new skelarn 'scorer' that uses our print_model_metrics function
def adjusted_f1(y_true, y_prob):
    f1 = print_model_metrics(y_true, y_prob, verbose = 0, return_metrics = True)[0]
    return f1
    
score = make_scorer(adjusted_f1, greater_is_better = True, needs_proba = True)



# Since we want to use a predefined Test/Val set, we'll use PredefinedSplit and pass it as the CV parameter
# We need to merge both the datasets and label 0 for test and -1 for the train set

X = sparse.vstack((train_features, test_features))
test_fold = [-1 for _ in range(train_features.shape[0])] + [0 for _ in range(test_features.shape[0])]
y = np.concatenate([y_train, y_test])
ps = PredefinedSplit(test_fold)

def run_grid_search(model, params, x_train, y_train):
    grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid.fit(x_train, y_train)
    return (grid.best_params_, grid.best_score_)


In [133]:
lr = SGDClassifier(loss = 'log')
lr_params = {'alpha' : [10**(-x) for x in range(7)],
             'penalty' : ['l1', 'l2', 'elasticnet'],
             'l1_ratio' : [0.15, 0.25, 0.5, 0.75]}
best_params, best_f1 = run_grid_search(lr, lr_params, X, y)
print('Best Parameters : {}'.format(best_params))
lr = SGDClassifier(loss = 'log', 
                   alpha = best_params['alpha'], 
                   penalty = best_params['penalty'], 
                   l1_ratio = best_params['l1_ratio'])
lr.fit(train_features, y_train)
y_test_prob = lr.predict_proba(test_features)[:,1]
print_model_metrics(y_test, y_test_prob)

Best Parameters : {'alpha': 1, 'l1_ratio': 0.15, 'penalty': 'l2'}
F1: 0.778 | Precision: 0.938 | Recall: 0.714 | AUC: 0.796 | Accuracy: 0.810 



In [139]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint

batch_size = 128
epochs = 30

simple_nn = Sequential()
simple_nn.add(Dense(150, activation='relu', input_shape=(112,)))
simple_nn.add(Dropout(0.2))
simple_nn.add(Dense(100, activation='relu'))
simple_nn.add(Dropout(0.2))
simple_nn.add(Dense(1, activation='sigmoid'))

simple_nn.summary()

simple_nn.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

checkpoint = ModelCheckpoint('./', monitor = 'val_accuracy', verbose = 1, save_best_only=True)
history = simple_nn.fit(train_features.todense(), y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks = [checkpoint],
                    validation_data=(test_features.todense(), y_test))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 150)               16950     
_________________________________________________________________
dropout_10 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 100)               15100     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 101       
Total params: 32,151
Trainable params: 32,151
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 0.6775 - accuracy: 0.5900
Epoch 00001: val_accurac

In [140]:
y_pred_prob = simple_nn.predict(test_features.todense())
print_model_metrics(y_test, y_pred_prob)

F1: 0.727 | Precision: 0.708 | Recall: 0.810 | AUC: 0.787 | Accuracy: 0.714 



In [141]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
svm = SVC(C = 10, kernel = 'poly', degree = 2, probability = True, verbose = 0)
svm_bag = BaggingClassifier(svm, n_estimators = 200, max_features = 0.9, max_samples = 1.0, bootstrap_features = False, bootstrap = True, n_jobs = 1, verbose = 0)
svm_bag.fit(train_features, y_train)
y_test_prob = svm_bag.predict_proba(test_features)[:,1]
print_model_metrics(y_test, y_test_prob)

F1: 0.667 | Precision: 0.923 | Recall: 0.571 | AUC: 0.751 | Accuracy: 0.738 



In [142]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.svm import SVC

# Define all models 
lr = SGDClassifier(loss = 'log', alpha = 0.1, penalty = 'elasticnet')
svm = SVC(C = 10, kernel = 'poly', degree = 2, probability = True)
nb = MultinomialNB(alpha = 10000, class_prior = [0.5, 0.5])
knn = KNeighborsClassifier(n_neighbors = 7, weights = 'distance', n_jobs = -1)
rf = RandomForestClassifier(n_estimators = 250, min_samples_split = 5, max_depth = 15,  n_jobs = -1)
xgb = XGBClassifier(n_estimators = 100, learning_rate = 0.3, max_depth = 1, n_jobs = -1)

model_dict = dict(zip(['LR', 'SVM', 'NB', 'KNN', 'RF', 'XGB'], [lr, svm, nb, knn, rf, xgb]))

for model_name, model in model_dict.items():
    print('Training {}'.format(model_name))
    model.fit(train_features, y_train)

model_weights = {   'LR' : 0.9,
                    'SVM' : 0.9,
                    'NB' : 0.8,
                    'KNN' : 0.75,
                    'RF' : 0.75,
                    'XGB' : 0.6,
                    'simple_nn' : 0.7
}

y_pred_prob = 0

for model_name, model in model_dict.items():
    y_pred_prob += (model.predict_proba(test_features)[:,1] * model_weights[model_name])

y_pred_prob += (simple_nn.predict(test_features.todense()).ravel() * model_weights['simple_nn'])
y_pred_prob /= sum(model_weights.values())

print_model_metrics(y_test, y_pred_prob)

Training LR
Training SVM
Training NB
Training KNN
Training RF
Training XGB
F1: 0.714 | Precision: 0.727 | Recall: 0.762 | AUC: 0.762 | Accuracy: 0.714 



In [143]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def run_voting_clf(model_weights):
    y_pred_prob = 0
    for model_name, model in model_dict.items():
        y_pred_prob += (model.predict_proba(test_features)[:,1] * model_weights[model_name])

    y_pred_prob += (simple_nn.predict(test_features.todense()).ravel() * model_weights['simple_nn'])
    y_pred_prob /= sum(model_weights.values())
    f1 = print_model_metrics(y_test, y_pred_prob, return_metrics = True, verbose = 0)[0]
    
    return {'loss' : -f1, 'status' : STATUS_OK}  #return negative of F1 since hyperopt is running fmin


trials = Trials()
model_weights = fmin(run_voting_clf,
    space= {
        'LR' : hp.uniform('LR', 0, 1),
        'SVM' : hp.uniform('SVM', 0, 1),
        'NB' : hp.uniform('NB', 0, 1),
        'KNN' : hp.uniform('KNN', 0, 1),
        'RF' : hp.uniform('RF', 0, 1),
        'XGB' : hp.uniform('XGB', 0, 1),
        'simple_nn' : hp.uniform('simple_nn', 0, 1),
    },
    algo=tpe.suggest,
    max_evals=500,
    trials = trials)

100%|██████████| 500/500 [02:16<00:00,  3.66it/s, best loss: -0.7727272727272727]


In [144]:
model_weights

{'KNN': 0.0024637601554303138,
 'LR': 0.9327498530577746,
 'NB': 0.11379765846571525,
 'RF': 0.605356274189167,
 'SVM': 0.017197885468432535,
 'XGB': 0.003689206246366962,
 'simple_nn': 0.1459630376644656}

In [149]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
selector = SelectKBest(k = 80)
train_features_selected = selector.fit_transform(train_features, y_train)
test_features_selected = selector.transform(test_features)
run_log_reg(train_features_selected, test_features_selected, y_train, y_test)

F1: 0.697 | Precision: 0.653 | Recall: 0.852 | AUC: 0.717 | Accuracy: 0.652 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [0] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [160]:
selector = SelectKBest(k = 45)
train_features_selected = selector.fit_transform(train_features, y_train)
test_features_selected = selector.transform(test_features)
run_log_reg(train_features_selected, test_features_selected, y_train, y_test, alpha = 1e-2)

F1: 0.703 | Precision: 0.843 | Recall: 0.657 | AUC: 0.770 | Accuracy: 0.743 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [0] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [157]:
selector = SelectPercentile(percentile = 35)
train_features_selected = selector.fit_transform(train_features, y_train)
test_features_selected = selector.transform(test_features)
run_log_reg(train_features_selected, test_features_selected, y_train, y_test, alpha = 1e-2)

F1: 0.712 | Precision: 0.808 | Recall: 0.705 | AUC: 0.764 | Accuracy: 0.733 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [0] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [151]:
np.array(feature_names)[selector.get_support()]

array(['longest_word_length', 'length_in_chars', 'sentiment_neg',
       'senitment_pos', 'sentiment_compound', 'glove_3', 'glove_4',
       'glove_9', 'glove_10', 'glove_17', 'glove_19', 'glove_24',
       'glove_26', 'glove_28', 'glove_32', 'glove_36', 'glove_38',
       'glove_40', 'glove_43', 'glove_45', 'glove_48', 'glove_50',
       'glove_56', 'glove_58', 'glove_60', 'glove_63', 'glove_64',
       'glove_68', 'glove_74', 'glove_76', 'glove_80', 'glove_81',
       'glove_82', 'glove_84', 'glove_85', 'glove_86', 'glove_87',
       'glove_88', 'glove_89', 'glove_94', 'glove_96', 'glove_98'],
      dtype='<U28')

In [171]:
from sklearn.feature_selection import RFECV
log_reg = SGDClassifier(loss = 'log', alpha = 1e-3)
selector = RFECV(log_reg, scoring = 'f1', n_jobs = -1, cv = ps, verbose = 1)
selector.fit(X, y)
# Now lets select the best features and check the performance
train_features_selected = selector.transform(train_features)
test_features_selected = selector.transform(test_features)
run_log_reg(train_features_selected, test_features_selected, y_train, y_test, alpha = 1e-1)

Fitting estimator with 112 features.
Fitting estimator with 111 features.
Fitting estimator with 110 features.
Fitting estimator with 109 features.
Fitting estimator with 108 features.
Fitting estimator with 107 features.
Fitting estimator with 106 features.
Fitting estimator with 105 features.
Fitting estimator with 104 features.
Fitting estimator with 103 features.
Fitting estimator with 102 features.
Fitting estimator with 101 features.
Fitting estimator with 100 features.
Fitting estimator with 99 features.
Fitting estimator with 98 features.
Fitting estimator with 97 features.
Fitting estimator with 96 features.
Fitting estimator with 95 features.
Fitting estimator with 94 features.
Fitting estimator with 93 features.
F1: 0.761 | Precision: 0.893 | Recall: 0.714 | AUC: 0.820 | Accuracy: 0.790 



SGDClassifier(alpha=0.1, average=False, class_weight=None, early_stopping=False,
              epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
              learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [172]:
print('Number of features selected:{}'.format(selector.n_features_))
np.array(feature_names)[selector.support_]

Number of features selected:92


array(['longest_word_length', 'mean_word_length', 'length_in_chars',
       'stop_words_ratio', 'hyperbolic_ratio', 'sentiment_neg',
       'senitment_pos', 'flesch_kincaid_grade',
       'dale_chall_readability_score', 'glove_1', 'glove_2', 'glove_4',
       'glove_5', 'glove_6', 'glove_7', 'glove_8', 'glove_9', 'glove_10',
       'glove_11', 'glove_12', 'glove_14', 'glove_15', 'glove_16',
       'glove_18', 'glove_19', 'glove_21', 'glove_22', 'glove_24',
       'glove_26', 'glove_27', 'glove_30', 'glove_31', 'glove_32',
       'glove_33', 'glove_34', 'glove_36', 'glove_39', 'glove_40',
       'glove_41', 'glove_42', 'glove_43', 'glove_44', 'glove_45',
       'glove_46', 'glove_47', 'glove_48', 'glove_49', 'glove_50',
       'glove_51', 'glove_53', 'glove_54', 'glove_55', 'glove_56',
       'glove_57', 'glove_59', 'glove_60', 'glove_61', 'glove_63',
       'glove_64', 'glove_65', 'glove_66', 'glove_67', 'glove_68',
       'glove_69', 'glove_70', 'glove_71', 'glove_74', 'glove_75',
   

In [185]:
# WHOA!
from mlxtend.feature_selection import SequentialFeatureSelector
log_reg = SGDClassifier(loss = 'log', alpha = 1e-2)
selector = SequentialFeatureSelector(log_reg, k_features = 'best', floating = True, cv = ps, scoring = 'f1', verbose = 1, n_jobs = -1) # k_features = 'best' returns the best subset of features
selector.fit(X.tocsr(), y)
train_features_selected = selector.transform(train_features.tocsr())
test_features_selected = selector.transform(test_features.tocsr())
run_log_reg(train_features_selected, test_features_selected, y_train, y_test, alpha = 1e-2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:    1.1s finished
Features: 1/112[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 111 out of 111 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 2/112[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 110 out of 110 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 3/112[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 109 out of 109 | elapsed:    0.3s f

F1: 0.900 | Precision: 0.950 | Recall: 0.905 | AUC: 0.922 | Accuracy: 0.905 



SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [183]:
# log_reg = SGDClassifier(loss='log', alpha=1e-4, n_jobs=-1, penalty='l2')
# log_reg.fit(x_train, y_train)
log_reg.fit(train_features_selected, y_train)
new_features, tst, ftr = featurize(test_set_copy, test_dataset, 'tfidf_glove')
new_features_selected = selector.transform(new_features.tocsr())
# x_t = tfidf_glove(test_set_copy)
predicted_output = log_reg.predict(new_features_selected)
output_string = np.where(predicted_output == 1, "Yes", "No")

test_set_copy_3 = test_set_copy.copy()
test_set_copy_3["useful_predicted"] = output_string
test_set_copy_3.drop(columns=["Useful", "snippet"], inplace=True)
test_set_copy_3

Text Features....
Word ratios....
Sentiment Scores....
Readability Scores....


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Glove.....


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


,name,url,snippet_actual,useful_predicted
0,International Leadership Development Programme...,https://www.faithandpraxis.org/go.php?a=2a&id=48,AIM OF THE PROGRAMME. To energise and equip me...,No
1,Leadership Management International,https://www.lmi-world.com/,"Empowering, rather than overpowering, is the w...",No
2,International Programs - The United States Cen...,https://www.census.gov/internationalprograms,International Programs The U.S. Census Bureau ...,No
3,"Visas for Foreign Government, International Or...",https://travel.state.gov/content/travel/en/us-...,If you are being sent by your government to a ...,No
4,AP State Portal,http://www.ap.gov.in/,AP Govt focus on enhancing the Quality of Life...,No
...,...,...,...,...
363,Study options and entry requirements | The Uni...,https://www.ed.ac.uk/global/study-abroad/study...,The University of Edinburgh is a charitable bo...,No
364,Cross Sector Leadership Exchange CSLE | Privat...,https://www.csleuk.org/,CSLE CIC deliver leadership programmes to enco...,No
365,Dr Gareth McGrath - Director of Parliamentary ...,https://uk.linkedin.com/in/dr-gareth-mcgrath-1...,Since 2011 I have co-delivered a political exc...,Yes
366,Programme content | MBA | London Business School,https://www.london.edu/masters-degrees/mba/pro...,In order to graduate from the programme at one...,No


In [0]:
test_set_copy_3.to_csv(os.path.join(BASE_DIR, "Rectified", "predicted_data.csv"), index=None)

In [166]:
print('Features selected {}'.format(len(selector.k_feature_idx_)))
np.array(feature_names)[list(selector.k_feature_idx_)]

Features selected 39


array(['easy_words_ratio', 'stop_words_ratio', 'contractions_ratio',
       'hyperbolic_ratio', 'sentiment_neg', 'flesch_kincaid_grade',
       'dale_chall_readability_score', 'glove_2', 'glove_5', 'glove_11',
       'glove_12', 'glove_14', 'glove_16', 'glove_21', 'glove_22',
       'glove_25', 'glove_26', 'glove_30', 'glove_33', 'glove_34',
       'glove_39', 'glove_41', 'glove_43', 'glove_52', 'glove_54',
       'glove_57', 'glove_61', 'glove_67', 'glove_68', 'glove_70',
       'glove_71', 'glove_72', 'glove_76', 'glove_85', 'glove_86',
       'glove_92', 'glove_93', 'glove_97', 'glove_99'], dtype='<U28')

In [168]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(50)
train_featurse_decomposed = svd.fit_transform(train_features)
test_featurse_decomposed = svd.transform(test_features)
run_log_reg(train_featurse_decomposed, test_featurse_decomposed, y_train, y_test, alpha = 1e-1)

F1: 0.744 | Precision: 0.928 | Recall: 0.671 | AUC: 0.793 | Accuracy: 0.786 



SGDClassifier(alpha=0.1, average=False, class_weight=None, early_stopping=False,
              epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
              learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)